In [1455]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

## Image Label (Neural Network)

In [1456]:
"""
2 LAYER

class Net(nn.Module):

    def __init__(self, INPUT_SIZE):
        super(Net, self).__init__()
        # Input is output of resnet18 image classification model
        self.fc1 = nn.Linear(INPUT_SIZE, 500) 
        self.fc11 = nn.Linear(500, 100)
        self.fc2 = nn.Linear(100, 1)
        self.act_out = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc11(x))
        x = self.dropout(x)
        x = self.act_out(self.fc2(x))
        return x
"""

'\n2 LAYER\n\nclass Net(nn.Module):\n\n    def __init__(self, INPUT_SIZE):\n        super(Net, self).__init__()\n        # Input is output of resnet18 image classification model\n        self.fc1 = nn.Linear(INPUT_SIZE, 500) \n        self.fc11 = nn.Linear(500, 100)\n        self.fc2 = nn.Linear(100, 1)\n        self.act_out = nn.Sigmoid()\n        self.dropout = nn.Dropout(p=0.25)\n\n    def forward(self, x):\n        x = F.relu(self.fc1(x))\n        x = self.dropout(x)\n        x = F.relu(self.fc11(x))\n        x = self.dropout(x)\n        x = self.act_out(self.fc2(x))\n        return x\n'

In [1457]:
"""
3 LAYER

class Net(nn.Module):

    def __init__(self, INPUT_SIZE):
        super(Net, self).__init__()
        # Input is output of resnet18 image classification model
        self.fc1 = nn.Linear(INPUT_SIZE, 750) 
        self.fc11 = nn.Linear(750, 250)
        self.fc2 = nn.Linear(250, 100)
        self.fc3 = nn.Linear(100, 1)
        self.act_out = nn.Sigmoid()

        self.dropout = nn.Dropout(p=0.25)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc11(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.act_out(self.fc3(x))
        return x
"""

'\n3 LAYER\n\nclass Net(nn.Module):\n\n    def __init__(self, INPUT_SIZE):\n        super(Net, self).__init__()\n        # Input is output of resnet18 image classification model\n        self.fc1 = nn.Linear(INPUT_SIZE, 750) \n        self.fc11 = nn.Linear(750, 250)\n        self.fc2 = nn.Linear(250, 100)\n        self.fc3 = nn.Linear(100, 1)\n        self.act_out = nn.Sigmoid()\n\n        self.dropout = nn.Dropout(p=0.25)\n        \n    def forward(self, x):\n        x = F.relu(self.fc1(x))\n        x = self.dropout(x)\n        x = F.relu(self.fc11(x))\n        x = self.dropout(x)\n        x = F.relu(self.fc2(x))\n        x = self.act_out(self.fc3(x))\n        return x\n'

In [1458]:
class Net(nn.Module):

    def __init__(self, INPUT_SIZE):
        super(Net, self).__init__()
        # Input is output of resnet18 image classification model
        self.fc1 = nn.Linear(INPUT_SIZE, 500) 
        self.fc11 = nn.Linear(500, 100)
        self.fc2 = nn.Linear(100, 1)
        self.act_out = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc11(x))
        x = self.dropout(x)
        x = self.act_out(self.fc2(x))
        return x


def load_dataset_df(label_path):
    label = pd.read_csv(label_path)
    label = label.drop(columns=['Unnamed: 0'])
    dataset = label

    # load embeddings
    with open('./dataset_work/embeddings.json') as f:
        embeddings = json.load(f)
    all_embeddings = pd.DataFrame.from_dict(embeddings, orient='index', columns=['embedding'])
    all_embeddings.index.name = 'id'
    dataset = pd.merge(dataset, all_embeddings, on='id', how='left')

    # load personalities
    with open('./dataset_work/personalities.json') as f:
        personalities = json.load(f)
    all_personalities = pd.DataFrame.from_dict(personalities, orient='index', columns=['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness'])
    all_personalities.index.name = 'id'
    dataset = pd.merge(dataset, all_personalities, on='id', how='left')

    return dataset

def vectorize_dataset(df, with_personalities, with_embeddings):
    if with_personalities and with_embeddings:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        embeddings = x[:,0]
        expanded = np.array(embeddings.tolist())
        x = np.concatenate((expanded, x[:,1:]), axis=1)
        x = np.array(x, dtype='float64')
        y = df['label'].values
    elif with_embeddings:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        embeddings = x[:,0]
        expanded = np.array(embeddings.tolist())
        x = expanded
        y = df['label'].values
    else:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        x = x[:,1:]
        x = np.array(x, dtype="float64")
        y = df['label'].values
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

def load(label_path, with_personalities, with_embeddings):
    df = load_dataset_df(label_path)
    return vectorize_dataset(df, with_personalities, with_embeddings)

def nn_pred(label_path):
    # load the weights, trained on only images
    # 3 layer train
    #PATH = "trained_models/BEST_MODEL_ALICE/alice_medium_images_True_personalities_False_optm_adam_loss_MSE_EPOCHS_100_BATCH_1000_REG_0.095_LR_0.0001_f1_0.788327619047619"
    # 2 layer train
    PATH = "trained_models/BEST_MODEL_ALICE/ALICE_2_LAYER_BEST_optm_adam_loss_MSE_EPOCHS_100_BATCH_1000_REG_0.09_LR_0.0001_f1_0.7920085561497326"
    train_x_t, train_y_t = load(label_path, with_personalities=False, with_embeddings=True)
    model = Net(train_x_t.shape[1])
    model.load_state_dict(torch.load(PATH))
    model.eval()
    return model(train_x_t)

In [1459]:
nn_preds_t = nn_pred('dataset_work/labels/image_only/alice_train_personalityFalse_imageTrue_labels.csv')
train_i = load_dataset_df('./dataset_work/labels/image_only/alice_train_personalityFalse_imageTrue_labels.csv')


In [1460]:
print(classification_report(nn_preds_t.round().detach().numpy(), train_i['label']))

              precision    recall  f1-score   support

         0.0       0.94      0.85      0.89       253
         1.0       0.73      0.89      0.80       122

    accuracy                           0.86       375
   macro avg       0.84      0.87      0.85       375
weighted avg       0.87      0.86      0.86       375



In [1461]:
nn_preds = nn_pred('dataset_work/labels/image_only/alice_valid_personalityFalse_imageTrue_labels.csv')
valid_i = load_dataset_df('./dataset_work/labels/image_only/alice_valid_personalityFalse_imageTrue_labels.csv')
#valid_i = load_dataset_df('./dataset_work/labels/image_and_personality/alice_valid_personalityFalse_imageTrue_labels.csv')

In [1462]:
print(classification_report(nn_preds.round().detach().numpy(), valid_i['label']))

              precision    recall  f1-score   support

         0.0       0.84      0.75      0.80        85
         1.0       0.57      0.70      0.63        40

    accuracy                           0.74       125
   macro avg       0.71      0.73      0.71       125
weighted avg       0.76      0.74      0.74       125



In [1488]:
test_i = load_dataset_df('./dataset_work/labels/image_only/alice_test_personalityFalse_imageTrue_labels.csv')
test_preds = nn_pred('dataset_work/labels/image_only/alice_test_personalityFalse_imageTrue_labels.csv')
print(classification_report(test_preds.round().detach().numpy(), test_i['label']))

              precision    recall  f1-score   support

         0.0       0.78      0.73      0.75        78
         1.0       0.59      0.65      0.62        46

    accuracy                           0.70       124
   macro avg       0.68      0.69      0.69       124
weighted avg       0.71      0.70      0.70       124



In [1463]:
len(nn_preds)

125

## Personality Label (Decision Trees)

In [1464]:
def load_dataset_p(label_path):
    label = pd.read_csv(label_path)
    label = label.drop(columns=['Unnamed: 0'])
    p_file = open('./dataset_work/personalities.json')
    personalities = json.load(p_file)
    p_file.close()
    all_personalities = pd.DataFrame.from_dict(personalities, orient='index', columns=['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness'])
    all_personalities['id'] = all_personalities.index
    dataset = pd.merge(label, all_personalities, on='id', how='left')
    return dataset

In [1465]:
# laod the personality datasets
train_p = load_dataset_p('./dataset_work/labels/personality_only/alice_train_personalityTrue_imageFalse_labels.csv')
valid_p = load_dataset_p('./dataset_work/labels/personality_only/alice_valid_personalityTrue_imageFalse_labels.csv')
test_p = load_dataset_p('./dataset_work/labels/personality_only/alice_test_personalityTrue_imageFalse_labels.csv')

In [1466]:
def split_dataset(d_df):
    _feats = d_df[['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
    x = _feats.values
    return x, d_df['label'].values

In [1467]:
X_train_p, y_train_p = split_dataset(train_p)
X_valid_p, y_valid_p = split_dataset(valid_p)
X_test_p, y_test_p = split_dataset(test_p)

In [1468]:
# add regularization with 'lambda'
personality_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, n_estimators=35, scale_pos_weight=0.8, reg_lambda=3)

#personality_model = GaussianNB() # likelihood of the features is assumed to be gaussian
personality_model.fit(X_train_p, y_train_p)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=35, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [1469]:
valid_preds = personality_model.predict(X_valid_p)
train_preds = personality_model.predict(X_train_p)
test_preds = personality_model.predict(X_test_p)
#p_probs = [v[1] for v in valid_preds] # probability of a output label '1'

In [1470]:
sum(y_valid_p) / len(y_valid_p)

0.752

In [1471]:
print(classification_report(train_preds, y_train_p))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        92
           1       0.98      0.98      0.98       283

    accuracy                           0.97       375
   macro avg       0.96      0.96      0.96       375
weighted avg       0.97      0.97      0.97       375



In [1472]:
print(classification_report(valid_preds, y_valid_p))

              precision    recall  f1-score   support

           0       0.84      0.72      0.78        36
           1       0.89      0.94      0.92        89

    accuracy                           0.88       125
   macro avg       0.87      0.83      0.85       125
weighted avg       0.88      0.88      0.88       125



In [1473]:
print(classification_report(test_preds, y_test_p))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84        32
           1       0.96      0.92      0.94        92

    accuracy                           0.91       124
   macro avg       0.88      0.90      0.89       124
weighted avg       0.92      0.91      0.91       124



## Putting them together on images + personalities

In [1474]:
def gen_final_data(nn_path, p_model, d_b, X_b, out):
    # create a 3-vector [breed, NN output, tree output] for the training set  
    nn_preds = nn_pred(nn_path)
    preds = p_model.predict_proba(X_b) # using the model trained on only personality data
    p_preds = [v[1] for v in preds]
    dog_ids = d_b['id'].values
    breed_ids = [i[1:i.find('_')] for i in dog_ids]
    
    dog_ids = d_b['id'].values
    breed_ids = [i[1:i.find('_')] for i in dog_ids]
    breed_to_a = {}
    ctr = 0
    for breed in breed_ids:
        if breed not in breed_to_a:
            breed_to_a[breed] = ctr
            ctr += 1

    processed_breed_ids = [breed_to_a[b] for b in breed_ids]
    
    out['breed'] = processed_breed_ids
    out['p_out'] = p_preds
    out['img_out'] = nn_preds.detach().numpy()
    

#     for breed in breed_to_a.keys():
#         nc = []
#         for i in breed_ids:
#             if i == breed:
#                 #print('y')
#                 nc.append(1)
#             else:
#                 nc.append(0)
#         out[breed] = nc
 

In [1475]:
train_b = load_dataset_df('./dataset_work/labels/image_and_personality/alice_train_personalityTrue_imageTrue_labels.csv')
valid_b = load_dataset_df('./dataset_work/labels/image_and_personality/alice_valid_personalityTrue_imageTrue_labels.csv')
test_b = load_dataset_df('./dataset_work/labels/image_and_personality/alice_test_personalityTrue_imageTrue_labels.csv')

X_train_b, y_train_b = split_dataset(train_b)
X_valid_b, y_valid_b = split_dataset(valid_b)
X_test_b, y_test_b = split_dataset(test_b)

In [1476]:
BOTH_TRAIN_PATH = './dataset_work/labels/image_and_personality/alice_train_personalityTrue_imageTrue_labels.csv'
BOTH_VALID_PATH = './dataset_work/labels/image_and_personality/alice_valid_personalityTrue_imageTrue_labels.csv'
BOTH_TEST_PATH = './dataset_work/labels/image_and_personality/alice_test_personalityTrue_imageTrue_labels.csv'
train_master = pd.DataFrame()
valid_master = pd.DataFrame()
test_master = pd.DataFrame()
gen_final_data(BOTH_TRAIN_PATH, personality_model, train_b, X_train_b, train_master)
gen_final_data(BOTH_VALID_PATH, personality_model, valid_b, X_valid_b, valid_master)
gen_final_data(BOTH_TEST_PATH, personality_model, test_b, X_test_b, test_master)

train_master_X = train_master.values
valid_master_X = valid_master.values
test_master_X = test_master.values

In [1477]:
test_master

,breed,p_out,img_out
0,0,0.985445,0.561628
1,1,0.042865,0.270346
2,2,0.796752,0.633707
3,3,0.947014,0.296303
4,4,0.980010,0.575307
...,...,...,...
119,23,0.978018,0.446371
120,3,0.286179,0.187637
121,13,0.668836,0.531577
122,24,0.997368,0.151803


In [1478]:
master_model = xgb.XGBClassifier(objective="binary:logistic",  booster='gbtree',random_state=42, n_estimators=20, scale_pos_weight=3, reg_lambda=2.4)
#master_model = LogisticRegression()
master_model.fit(train_master_X, y_train_b)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=20, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [1479]:
sum(y_train_b) / len(y_train_b)

0.41333333333333333

In [1480]:
final_train_preds = master_model.predict(train_master_X)

In [1481]:
print(classification_report(y_train_b, final_train_preds))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92       220
           1       0.83      1.00      0.91       155

    accuracy                           0.91       375
   macro avg       0.91      0.93      0.91       375
weighted avg       0.93      0.91      0.92       375



In [1482]:
final_valid_preds = master_model.predict(valid_master_X)

In [1483]:
print(classification_report(y_valid_b, final_valid_preds))

              precision    recall  f1-score   support

           0       0.77      0.68      0.72        79
           1       0.55      0.65      0.59        46

    accuracy                           0.67       125
   macro avg       0.66      0.67      0.66       125
weighted avg       0.69      0.67      0.68       125



In [1484]:
final_test_preds = master_model.predict(test_master_X)

In [1485]:
sum(final_test_preds) / len(final_test_preds)

0.5564516129032258

In [1486]:
print(classification_report(y_test_b, final_test_preds))

              precision    recall  f1-score   support

           0       0.89      0.65      0.75        75
           1       0.62      0.88      0.73        49

    accuracy                           0.74       124
   macro avg       0.76      0.77      0.74       124
weighted avg       0.79      0.74      0.74       124



In [1404]:
# Vrushank to-do: 
# refactor data, use k-fold